In [49]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import datetime as dt # For defining dates
import mplfinance as mpf # Matplotlib finance

import time

# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

## Dates & Other Constants

In [50]:
# Define path to files
# For MacOS
PATH = "/Users/oppoudel/dev/Python4Finance/Stocks/"
# For Windows
# PATH = "D:/Python for Finance/Stocks/"

# Start date defaults
S_YEAR = 2017
S_MONTH = 1
S_DAY = 3
S_DATE_STR = "2017-01-03"
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)

# End date defaults
E_YEAR = 2021
E_MONTH = 8
E_DAY = 19
E_DATE_STR = "2021-08-19"
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)

## Get Stock File Names in a List

In [51]:
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers

# On MacOS Only
tickers.sort()

## Function that Returns a Dataframe from a CSV

In [52]:
def get_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + '.csv')
    except FileNotFoundError:
        pass
        # print("File Doesn't Exist")
    else:
        return df

## Function that Saves Dataframe to CSV

In [53]:
def save_dataframe_to_csv(df, ticker):
    df.to_csv(PATH + ticker + '.csv')

## Return 1st Valid Date in Dataframe

In [54]:
def get_valid_dates(df, sdate, edate):  
    try:
        mask = (df['Date'] > sdate) & (df['Date'] <= edate) 
        sm_df = df.loc[mask]
        sm_df = sm_df.set_index(['Date'])
        sm_date = sm_df.index.min()
        last_date = sm_df.index.max()
        date_leading = '-'.join(('0' if len(x)<2 else '')+x for x in sm_date.split('-'))
        date_ending = '-'.join(('0' if len(x)<2 else '')+x for x in last_date.split('-'))
    except Exception:
        print("Date Corrupted")
    else:
        return date_leading, date_ending

## Returns Return on Investment over Time

In [55]:
def roi_between_dates(df, sdate, edate):
    try: 
        start_val = df.loc[sdate,'Adj Close'] 
        end_val = df.loc[edate,'Adj Close']
        roi = ((end_val - start_val) / start_val)
    except Exception:
        print("Data Corrupted")
    else:
        return roi

## Get Mean Between Dates

In [56]:
def get_mean_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    return df.loc[mask]["Adj Close"].mean()

## Get Standard Deviation Between Dates

In [57]:
def get_sd_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    return df.loc[mask]["Adj Close"].std()

## Get Coefficient of Variation Between Dates

In [58]:
def get_cov_between_dates(df, sdate, edate):
    mean = get_mean_between_dates(df, sdate, edate)
    sd = get_sd_between_dates(df, sdate, edate)
    return sd / mean

## Merge Multiple Stocks in One Dataframe by Column Name

In [59]:
def merge_df_by_column_name(col_name, sdate, edate, *tickers):
    # Will hold data for all dataframes with the same column name
    mult_df = pd.DataFrame()
    
    for x in tickers:
        df = get_df_from_csv(x)
        df['Date'] = pd.to_datetime(df['Date'])
        # Use a mask to grab data between defined dates
        mask = (df['Date'] >= sdate) & (df['Date'] <= edate)
        mult_df[x] = df.loc[mask][col_name]
        
    return mult_df

## 11 Stock Market Sectors

**Technology** : Manufacturing of electronics, software, or related to information technology 

**Health Care** : Biotech, hospitals, medical devices, drugs 

**Financials** : Banks, investment funds, and insurance

**Real Estate** : Residential, industrial, and retail real estate 

**Energy** : Production and supply of energy 

**Materials** : Mining, refining, chemical, and forestry 

**Consumer Discretionary** : Retailers, apparel, media, durables and services 

**Industrials** : Construction, machinery, fabrication, manufacturing, defense, and aerospace 

**Utilities** : Direct providers of electric, gas, and water 

**Consumer Staples** : Food, beverage as well as products consumers deem essential 

**Telecommunication** : Cable, internet providers, wireless, and satellite

## Get Sector Stocks

In [60]:
# For Windows
# sec_df = pd.read_csv('D:/Python for Finance/stock_sectors.csv')
# For Mac
sec_df = pd.read_csv('/Users/oppoudel/dev/Python4Finance/stock_sectors.csv')
sec_df

# Get Industrials DF
indus_df = sec_df.loc[sec_df['Sector'] == "Industrials"]
health_df = sec_df.loc[sec_df['Sector'] == "Health Care"]
it_df = sec_df.loc[sec_df['Sector'] == "Information Technology"]
comm_df = sec_df.loc[sec_df['Sector'] == "Communication Services"]
staple_df = sec_df.loc[sec_df['Sector'] == "Consumer Staples"]
discretion_df = sec_df.loc[sec_df['Sector'] == "Consumer Discretionary"]
utility_df = sec_df.loc[sec_df['Sector'] == "Utilities"]
financial_df = sec_df.loc[sec_df['Sector'] == "Financials"]
material_df = sec_df.loc[sec_df['Sector'] == "Materials"]
restate_df = sec_df.loc[sec_df['Sector'] == "Real Estate"]
energy_df = sec_df.loc[sec_df['Sector'] == "Energy"]


## Returns a DF with ROIs for all Stocks

In [92]:
def get_rois_for_stocks(stock_df):
    # Will hold all tickers & stock rois
    tickers = []
    rois = []

    # iterrows provides the index and column for each row in the DF
    for index, row in stock_df.iterrows():
        df = get_df_from_csv(row['Symbol'])
    
        # If we can't find the ticker delete it from the dataframe
        if df is None:
            stock_df = stock_df.drop(index)
        else:
            tickers.append(row['Symbol'])
            sdate, edate = get_valid_dates(df, '2021-01-01', '2021-12-31')
            df = df.set_index(['Date'])
            roi = roi_between_dates(df, sdate, edate)
            rois.append(roi)
    return pd.DataFrame({'Ticker':tickers, 'ROI':rois})

In [93]:
industrial = get_rois_for_stocks(indus_df)
health_care = get_rois_for_stocks(health_df)
it = get_rois_for_stocks(it_df)
commun = get_rois_for_stocks(comm_df)
staple = get_rois_for_stocks(staple_df)
discretion = get_rois_for_stocks(discretion_df)
utility = get_rois_for_stocks(utility_df)
finance = get_rois_for_stocks(financial_df)
material = get_rois_for_stocks(material_df)
restate = get_rois_for_stocks(restate_df)
energy = get_rois_for_stocks(energy_df)

## Top Industrials

In [95]:
industrial.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
24,GNRC,0.744968
48,RHI,0.676882
34,JCI,0.594923
45,PNR,0.533730
56,TXT,0.506829


## Top Healthcare

In [82]:
health_care.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
15,CTLT,2.366872
24,DXCM,2.209932
58,WST,1.962799
32,IDXX,1.739614
3,ALGN,1.643875


## Top Information Tech

In [83]:
it.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
21,ENPH,34.664634
2,AMD,3.870420
47,NVDA,2.854826
61,TER,2.825196
50,PAYC,2.778196


## Top Communications

In [84]:
commun.sort_values(by=['ROI'], ascending=False).head(20)

,Ticker,ROI
4,CHTR,1.297128
17,TMUS,1.066350
13,NFLX,1.020212
10,FB,1.013267
0,ATVI,1.003220
18,TTWO,0.997789
20,TWTR,0.879556
9,EA,0.789117
19,DIS,0.682979
2,GOOG,0.675078


## Top Staples

In [85]:
staple.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
10,EL,1.051833
9,COST,0.923046
22,MNST,0.896637
7,CAG,0.801563
11,GIS,0.647460


## Top Discretionary

In [86]:
discretion.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
52,TSLA,10.377370
42,PENN,3.440617
17,ETSY,2.785319
10,CMG,2.127729
51,TGT,1.797055


## Top Utilities

In [87]:
utility.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
18,NEE,0.899435
4,AWK,0.787637
0,AES,0.770789
25,SO,0.533027
27,XEL,0.457615


## Top Finance

In [88]:
finance.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
39,MSCI,2.100301
34,MKTX,1.760462
37,MCO,1.098858
51,SIVB,0.980341
49,SPGI,0.970903


## Top Materials

In [89]:
material.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
10,FCX,1.567112
3,BLL,1.119669
1,ALB,0.959996
21,SHW,0.907899
16,NEM,0.839971


## Top Real Estate

In [90]:
restate.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
8,EQIX,1.112940
17,PLD,0.841405
21,SBAC,0.771372
0,ARE,0.688282
7,DRE,0.684079


## Top Energy

In [108]:
energy.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
4,DVN,0.608962
11,MRO,0.580850
5,FANG,0.410011
15,OKE,0.392985
12,MPC,0.372723


## Portfolio List

In [96]:
port_list = ["GNRC", "DXCM", "AMD", "NFLX", "COST", "TGT", "AES", "MSCI", 
             "NEM", "SBAC", "HES"]

## Check for Correlation

In [97]:
mult_df = merge_df_by_column_name('daily_return',  '2018-01-01', 
                                  '2020-12-31', *port_list)
mult_df

# Generate a Correlation Matrix
mult_df.corr()

,GNRC,DXCM,AMD,NFLX,COST,TGT,AES,MSCI,NEM,SBAC,HES
GNRC,1.000000,0.289526,0.395187,0.340732,0.465304,0.338411,0.433325,0.515742,0.250697,0.444762,0.426870
DXCM,0.289526,1.000000,0.328297,0.307643,0.265366,0.196806,0.218423,0.469509,0.147494,0.243462,0.243240
AMD,0.395187,0.328297,1.000000,0.459581,0.394315,0.340573,0.258974,0.485819,0.191355,0.294906,0.288337
NFLX,0.340732,0.307643,0.459581,1.000000,0.430674,0.259971,0.204159,0.449132,0.103639,0.216935,0.217857
COST,0.465304,0.265366,0.394315,0.430674,1.000000,0.500433,0.319886,0.482322,0.183141,0.455022,0.234526
TGT,0.338411,0.196806,0.340573,0.259971,0.500433,1.000000,0.305004,0.325772,0.092757,0.304365,0.245106
AES,0.433325,0.218423,0.258974,0.204159,0.319886,0.305004,1.000000,0.483399,0.176918,0.333249,0.470414
MSCI,0.515742,0.469509,0.485819,0.449132,0.482322,0.325772,0.483399,1.000000,0.258932,0.498281,0.411647
NEM,0.250697,0.147494,0.191355,0.103639,0.183141,0.092757,0.176918,0.258932,1.000000,0.275681,0.149387
SBAC,0.444762,0.243462,0.294906,0.216935,0.455022,0.304365,0.333249,0.498281,0.275681,1.000000,0.273548


In [98]:
# Get the number of samples
days = len(mult_df.index)
days

756

In [99]:
def get_stock_price_on_date(ticker, date):
    df = get_df_from_csv(ticker)
    df = df.set_index(['Date'])
    return df.loc[date,'Adj Close']

In [102]:
# 01-04-2021 is the 1st date in 2021
print("GNRC $", get_stock_price_on_date('GNRC', '2021-01-04')) # 2 449.12
print("DXCM $", get_stock_price_on_date('DXCM', '2021-01-04')) # 1 357.74
print("AMD $", get_stock_price_on_date('AMD', '2021-01-04')) # 4 369.2
print("NFLX $", get_stock_price_on_date('NFLX', '2021-01-04')) # 1 522.86
print("COST $", get_stock_price_on_date('COST', '2021-01-04')) # 1 377.88
print("TGT $", get_stock_price_on_date('TGT', '2021-01-04')) # 3 527.46
print("AES $", get_stock_price_on_date('AES', '2021-01-04')) # 18 410.76
print("MSCI $", get_stock_price_on_date('MSCI', '2021-01-04')) # 1 435.40
print("NEM $", get_stock_price_on_date('NEM', '2021-01-04')) # 7 434.49
print("SBAC $", get_stock_price_on_date('SBAC', '2021-01-04')) # 2 548.84
print("HES $", get_stock_price_on_date('HES', '2021-01-04')) # 9 472.23

# Total Investment
tot_inv = 449.12 + 357.74 + 369.2 + 522.86 + 377.88 + 527.46 + 410.76 + \
435.40 + 434.49 + 548.84 + 472.23
tot_inv

GNRC $ 224.5599975585937
DXCM $ 357.739990234375
AMD $ 92.3000030517578
NFLX $ 522.8599853515625
COST $ 377.8815307617188
TGT $ 175.82254028320312
AES $ 22.82006454467773
MSCI $ 435.3967590332031
NEM $ 62.0733871459961
SBAC $ 274.4242248535156
HES $ 52.46865844726562


4905.980000000001

## Returns Portfolio Weight

In [103]:
def get_port_weight(price, total):
    return price / total

In [104]:
gnrc_w = get_port_weight(449.12, 4905.98)
dxcm_w = get_port_weight(357.74, 4905.98)
amd_w = get_port_weight(369.2, 4905.98)
nflx_w = get_port_weight(522.86, 4905.98)
cost_w = get_port_weight(377.88, 4905.98)
tgt_w = get_port_weight(527.46, 4905.98)
aes_w = get_port_weight(410.76, 4905.98)
msci_w = get_port_weight(435.40, 4905.98)
nem_w = get_port_weight(434.49, 4905.98)
sbac_w = get_port_weight(548.84, 4905.98)
hes_w = get_port_weight(472.23, 4905.98)

wts = np.array([gnrc_w, dxcm_w, amd_w, nflx_w, cost_w, tgt_w, aes_w, msci_w, \
                nem_w, sbac_w, hes_w])
wts

port_var = np.dot(wts.T, np.dot(mult_df.cov() * 756, wts))
print("Portfolio Covariance :", port_var)
print("GNRC Var :", mult_df["GNRC"].var() * 756)
print("NFLX Var :", mult_df["NFLX"].var() * 756)

Portfolio Covariance : 0.18221033377623255
GNRC Var : 0.43946543607242505
NFLX Var : 0.547725362304338


## Different Types of Risk

There is risk that you can limit through diversifaction (Idiosyncratic) and risk that you can't (Systematic). Systematic risk 
is caused by unforeseen conditions such as wars, recessions, natural disasters, dramatic changes in consumer actions, etc.

Data tells us that if we make a portfolio made up of approximately 25 stocks that aren't correlated that we can dramatically
lower idiosyncratic risk. That is one reason why people invest in indexes. You can further lower risk by investing in
other countries, bonds and cash.

## Diversifiable Risk

In [105]:
# Diversifiable Risk = Portfolio Variance - All Weighted Variances
def calc_diversifiable_risk(df, tickers, weights):
    # Gets number of days
    days = len(mult_df.index)
    # Calculate covariance of portfolio
    port_covar = np.dot(weights.T, np.dot(df.cov() * days, weights)) 
    
    i = 0
    while i < len(tickers):
        wt_sq = weights[i] ** 2
        stk_var = mult_df[tickers[i]].var() * days
        wt_var = wt_sq * stk_var
        port_covar = port_covar - wt_var
        i += 1
    return port_covar

In [106]:
div_risk = calc_diversifiable_risk(mult_df, port_list, wts)
div_risk

0.13626654511704744

## Find Risk that Can't be Diversified

In [107]:
print("Systematic Risk :", (port_var - div_risk))

Systematic Risk : 0.04594378865918511
